## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Изучим общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.tail(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости


In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
# проверим на взаимосвязь по возрасту клиента 
display(data[(data['total_income'].isna() == True) & (data['days_employed'].isna() == True)]['dob_years'].value_counts())

# проверим на взаимосвязь по цели кредита 
display(data[(data['total_income'].isna() == True) & (data['days_employed'].isna() == True)]['purpose'].value_counts())

# проверим на взаимосвязь по роду деятельности
data[(data['total_income'].isna() == True) & (data['days_employed'].isna() == True)]['income_type'].value_counts()

# явной зависимости пропусков от возраста, цели кредита или рода деятельности не обнаружил.

34    69
40    66
42    65
31    65
35    64
36    63
47    59
41    59
30    58
28    57
58    56
57    56
54    55
56    54
38    54
52    53
37    53
33    51
50    51
39    51
29    50
43    50
49    50
51    50
45    50
46    48
55    48
48    46
44    44
53    44
60    39
62    38
61    38
64    37
32    37
23    36
27    36
26    35
59    34
63    29
25    23
24    21
66    20
65    20
21    18
22    17
67    16
0     10
68     9
71     5
20     5
69     5
70     3
72     2
19     1
73     1
Name: dob_years, dtype: int64

на проведение свадьбы                     92
сыграть свадьбу                           81
свадьба                                   76
строительство собственной недвижимости    75
операции с жильем                         74
покупка недвижимости                      72
операции со своей недвижимостью           71
покупка жилья для семьи                   71
ремонт жилью                              70
операции с коммерческой недвижимостью     70
покупка коммерческой недвижимости         67
покупка жилья для сдачи                   65
недвижимость                              62
операции с недвижимостью                  61
покупка жилой недвижимости                61
жилье                                     60
строительство недвижимости                59
автомобили                                57
заняться высшим образованием              56
заняться образованием                     55
сделка с подержанным автомобилем          54
на покупку своего автомобиля              53
свой автом

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [6]:
total_income_median = data['total_income'].median()
days_employed_mean = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_mean)
data['total_income'] = data['total_income'].fillna(total_income_median)
display(data.isna().sum())
data.describe()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,56557.335698,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,134922.319298,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-385.106616,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Обнаружил пропущенные значения в столбцах "total_income" и "days_employed". Исходя из того что эти показатели одни из ключевых для оценки со стороны банка и необходимы мне для получения более корректных выводов мы не можем воспользоваться самым простым способом и проставить везде нулевые значения. Удалить тоже было бы некорректно на мой взгляд - пропусков по отношению ко всему стобцу более 10%. Остается заполнение медианой и средним арифметическим значением. К стажу применил среднее арифметическое, а к доходу медиану - она защищает от ошибочных или слишком больших/маленьких значений. Можно предположить, что пропуски оставили клиенты которые не были трудоустроены официально, но у них разные профессии и судя по анализу большинство из них вполне корпоративные.  Попробовал найти привязку к определенной возрастной категории, но никай закономерности не обнаружил - соответственно смысла высчитывать среднее значение и медиану для каждой отдельной категории смысла нет. Исходя из проведенного анализа делаю вывод, что ошибки являются случайными и выбранный мной способ оптимальный. 

### Замена типа данных

In [7]:
data['days_employed'] = data['days_employed'].astype('int32')
data['total_income'] = data['total_income'].astype('int32')
data['children'] = data['children'].astype('int8')
data['dob_years'] = data['dob_years'].astype('int8')
data['education_id'] = data['education_id'].astype('int8')
data['family_status_id'] = data['family_status_id'].astype('int8')
data['debt'] = data['debt'].astype('int8')
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int32
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int8
total_income        21525 non-null int32
purpose             21525 non-null object
dtypes: int32(2), int8(5), object(5)
memory usage: 1.1+ MB


В применении метода to_numeric() нет необходимости, т.к. данные уже типа float.

### Обработка дубликатов

In [9]:
Counter(data['education'])
data ['education'] = data ['education'].str.lower()
Counter(data['education'])

Counter(data['family_status'])
data ['family_status'] = data ['family_status'].str.lower()
Counter(data['family_status'])

Counter(data['income_type'])

Counter(data['purpose'])

def plus(value):
    if value < 0:
        value *= -1
        return value
data['days_employed'] = data['days_employed'].apply(plus)
# смотрим кол-во данных перед удалением
data.duplicated().sum()
data[data.duplicated(keep=False)].sort_values(by = ['days_employed', 'total_income'])
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

### Вывод

Привел строки к нижнему регистру и взял по модулю отрицательные значения в столбце "days_employed" с целью более корректного поиска дубликатов. Воспользовался единственным на данный момент знакомым методом удаления дубликатов.
На мой взгляд причина появления дубликатов - внесение в  итоговую таблицу данных из нескольких источников или файлов.

In [10]:
unique_purposes = data['purpose'].unique().tolist()
unique_purposes

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

In [11]:
m = Mystem()
# склейкой переведем список в строку
string = ' '.join(unique_purposes)
lemmas = m.lemmatize(string)
print(Counter(lemmas)) 

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Узнал кол-во уникальных значений, воспользовался библиотекой с функцией лемматизации на русском языке —
pymystem3, для подсчёта встречаемости значений в списке использую специальный контейнер Counter из модуля collections. Для последующей категоризации стоит выделить 4 леммы, отражающие причины получения займа: недвижимость/жилье, автомобиль, образование, свадьба.

### Категоризация данных

In [12]:
def purpose_id(purpose):
    try:
        if 'недвиж' in purpose:
            return 'недвижимость' 
        if 'жил' in purpose:
            return 'недвижимость'
        if 'автом' in purpose:
            return 'автомобиль'
        if 'образ' in purpose:
            return 'образование'
        if 'свад' in purpose:  
            return 'свадьба'
    except:
        return 'Ошибка'
    
data['purpose_id'] = data['purpose'].apply(purpose_id)
data['purpose_id'].value_counts()


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_id, dtype: int64

In [13]:
data.sort_values(by = 'purpose', ascending = False)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
20419,1,3508.0,65,высшее,0,гражданский брак,1,F,сотрудник,0,146556,сыграть свадьбу,свадьба
17734,0,NaN,65,среднее,1,гражданский брак,1,F,пенсионер,0,62808,сыграть свадьбу,свадьба
13181,0,3716.0,42,среднее,1,гражданский брак,1,M,сотрудник,0,94823,сыграть свадьбу,свадьба
18935,0,7782.0,56,высшее,0,гражданский брак,1,F,компаньон,0,131553,сыграть свадьбу,свадьба
8933,1,181.0,44,высшее,0,гражданский брак,1,F,сотрудник,0,196983,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16346,0,1763.0,30,высшее,0,не женат / не замужем,4,F,компаньон,0,372278,автомобили,автомобиль
19733,1,445.0,34,высшее,0,женат / замужем,0,M,компаньон,0,220920,автомобили,автомобиль
15067,0,3736.0,46,среднее,1,женат / замужем,0,F,сотрудник,1,90878,автомобили,автомобиль
11964,1,245.0,41,среднее,1,женат / замужем,0,M,компаньон,1,64110,автомобили,автомобиль


Категоризоровал причины получения займа и добавил этот столбец в таблицу. 

### Анализ полученных данных 

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [14]:
#Заменим отрицательные значения на положительные
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
# категоризируем кол-во детей в семье

def children_id(children):
    if children < 1:
        return '0 детей'
    if children == 1:
        return '1 ребенок'
    if children > 2:
        return '2 и более'
data['children_id'] = data['children'].apply(children_id)
data
data_pivot = data.pivot_table(index = ['children_id'], columns = 'debt', values = 'purpose', aggfunc = 'count')
data_pivot['ratio'] = data_pivot[1] / data_pivot[0]
data_pivot

debt,0,1,ratio
children_id,,,
0 детей,13028,1063,0.081593
1 ребенок,4410,445,0.100907
2 и более,349,31,0.088825


Избавился от нескольких неадекватных значений в столбце children и получил данные по зависимости просрочек платежей от кол-ва детей в семье. Разница небольшая, но семьи с детьми допускают просрочки платежей чаще бездетных семей. Стоит отметить, что семьи с одним ребенком допускают просрочки чаще семей где вообще нет детей, либо где их больше двух.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [15]:
data_pivot = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'purpose', aggfunc = 'count')
data_pivot['ratio'] = data_pivot[1] / data_pivot[0]
data_pivot

debt,0,1,ratio
family_status,,,
в разводе,1110,85,0.076577
вдовец / вдова,896,63,0.070312
гражданский брак,3763,388,0.103109
женат / замужем,11408,931,0.081609
не женат / не замужем,2536,274,0.108044


Судя по нашей выборке люди когда - либо бывшие или находящиеся на данный момент в браке имеют меньшее количество просрочек по кредиту.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [16]:
# разобъем на категории уровень месячного дохода
data['total_income'].median()
def income_id(total_income):
    if total_income <= 100000.0:
        return 'Низкий уровень дохода'
    if total_income <= 200000.0:
        return 'Средний уровень дохода'
    if total_income < 250000.0:
        return 'Высокий уровень дохода'
    return 'Короли'
data['income_id'] = data['total_income'].apply(income_id)
data['income_id'].value_counts()
data_pivot = data.pivot_table(index = ['income_id'], columns = 'debt', values = 'purpose', aggfunc = 'count')
data_pivot['ratio'] = data_pivot[1] / data_pivot[0]
data_pivot

debt,0,1,ratio
income_id,,,
Высокий уровень дохода,2090,164,0.078469
Короли,2618,194,0.074102
Низкий уровень дохода,4109,354,0.086152
Средний уровень дохода,10896,1029,0.094438


In [17]:
data['total_income'].describe()

count    2.145400e+04
mean     1.652256e+05
std      9.802102e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.450170e+05
75%      1.958132e+05
max      2.265604e+06
Name: total_income, dtype: float64

Выше уровень доходов - меньше вероятность просрочки платежа.

- Как разные цели кредита влияют на его возврат в срок?

In [18]:
data_pivot = data.pivot_table(index = ['purpose_id'], columns = 'debt', values = 'purpose', aggfunc = 'count')
data_pivot['ratio'] = data_pivot[1] / data_pivot[0]
data_pivot

debt,0,1,ratio
purpose_id,,,
автомобиль,3903,403,0.103254
недвижимость,10029,782,0.077974
образование,3643,370,0.101565
свадьба,2138,186,0.086997


Ответственней всего к выплатам по кредиту относятся люди, занявшие деньги на улучшение жилищных условий и решившие зарегистрировать отношения, что вполне естественно.

### Общий вывод

Самый рискованный для банка вариант - оформить кредит на автомобиль многодетному, никогда не бывшему в браке человеку с высоким доходом.
Я бы порекомендовал ограничиться только ипотеками овдовевшим бездетным королям.
Также разработать систему присваивания индивидуального номера клиенту с целью исключения возможности создания дубликатов. 
Исключить возможность заполнения форм заведомо ошибочной информацией (отрицательные и аномально высокие/низкие значения.)
Для групп заемщиков с высоким риском просрочки платежа (семьи с детьми, люди никогда не состоявшие в браке, заем с целью покупки автомобиля) предусмотреть дополнительное обеспечение в зависимости от суммы займа: поручительство юридического или физического лица, залог собственности.
Считаю необходимым провести дополнительный анализ с выявлением зависимости кол-ва просрочек платежей от типа организации - работодателя и частоты смены организации - работодателя за последние 5 лет.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [х]  удалены дубликаты;
- [х]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [х]  описаны возможные причины появления дубликатов в данных;
- [х]  выделены леммы в значениях столбца с целями получения кредита;
- [х]  описан процесс лемматизации;
- [х]  данные категоризированы;
- [х]  есть объяснение принципа категоризации данных;
- [х]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [х]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [х]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [х]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [х]  в каждом этапе есть выводы;
- [х]  есть общий вывод.